In [1]:
import pandas as pd

In [2]:
matching_df = pd.read_csv("output/cellprofiler_matching_map.csv")

In [3]:
print("Do compounds that target BRD4 look similar?")
matching_df.query('Metadata_matching_target=="BRD4" and Modality=="compound"')[
    ["Modality", "Cell", "time", "Metadata_matching_target", "above_corrected_p_threshold"]
].reset_index(drop=True).sort_values(by=["Modality", "Cell", "time"])

Do compounds that target BRD4 look similar?


,Modality,Cell,time,Metadata_matching_target,above_corrected_p_threshold
1,compound,A549,long,BRD4,False
0,compound,A549,short,BRD4,False
3,compound,U2OS,long,BRD4,False
2,compound,U2OS,short,BRD4,False


In [4]:
cosine_sim = pd.read_csv("output/compound_genetic_perturbation_cosine_similarity.csv")

metadata = (
    pd.read_csv(
        "../metadata/external_metadata/JUMP-Target-1_compound_metadata.tsv",
        sep="\t",
        usecols=["broad_sample", "pert_iname"],
    )
    .merge(
        pd.read_csv(
            "input/JUMP-Target-1_compound_metadata_additional_annotations.tsv",
            sep="\t",
            usecols=["broad_sample", "moa_list"],
        )
    )
    .rename(columns={"broad_sample": "Metadata_broad_sample"})
)

In [5]:
cosine_sim_brd4 = cosine_sim.query('Metadata_matching_target=="BRD4"').merge(
    metadata, on="Metadata_broad_sample"
)
cosine_sim_bi2536 = cosine_sim.merge(metadata, on="Metadata_broad_sample").query(
    'pert_iname=="BI-2536"'
)

In [6]:
print("Cosine similarity of BI-2536 to its gene targets")
cosine_sim_bi2536.sort_values(by="cosine_sim", ascending=False).groupby(
    ["Metadata_matching_target", "Genetic_Perturbation", "Cell"]
).cosine_sim.median().reset_index()

Cosine similarity of BI-2536 to its gene targets


,Metadata_matching_target,Genetic_Perturbation,Cell,cosine_sim
0,BRD4,CRISPR,A549,0.377306
1,BRD4,CRISPR,U2OS,0.111332
2,BRD4,ORF,A549,-0.233175
3,BRD4,ORF,U2OS,0.600489
4,PLK1,CRISPR,A549,0.755886
5,PLK1,CRISPR,U2OS,0.787449
6,PLK1,ORF,A549,0.106761
7,PLK1,ORF,U2OS,-0.026110


In [7]:
print("Cosine similarity of BRD4 to the compounds that target it")
cosine_sim_brd4.sort_values(by="cosine_sim", ascending=False).groupby(
    ["pert_iname", "Genetic_Perturbation", "Cell"]
).cosine_sim.median().reset_index()

Cosine similarity of BRD4 to the compounds that target it


,pert_iname,Genetic_Perturbation,Cell,cosine_sim
0,BI-2536,CRISPR,A549,0.377306
1,BI-2536,CRISPR,U2OS,0.111332
2,BI-2536,ORF,A549,-0.233175
3,BI-2536,ORF,U2OS,0.600489
4,PFI-1,CRISPR,A549,0.394729
5,PFI-1,CRISPR,U2OS,-0.043047
6,PFI-1,ORF,A549,-0.176815
7,PFI-1,ORF,U2OS,0.306706
8,TG-101348,CRISPR,A549,0.045648
9,TG-101348,CRISPR,U2OS,0.149345


Check if PFI-1 and TG-101348 are replicable

In [8]:
other_brd4 = metadata.query('pert_iname=="PFI-1" or pert_iname=="TG-101348"')

In [9]:
replicate_retrieval_other_brd4 = pd.read_csv("output/cellprofiler_replicability_map.csv").merge(
    other_brd4, on="Metadata_broad_sample"
)
replicate_retrieval_other_brd4

,Metadata_broad_sample,mean_average_precision,nlog10pvalue,corrected_p_value,nlog10correctedpvalue,above_p_threshold,above_corrected_p_threshold,Description,Modality,Cell,time,timepoint,pert_iname,moa_list
0,BRD-K12502280-001-11-4,1.000000,5.000004,0.000033,4.482766,True,True,compound_A549_short,compound,A549,short,24,TG-101348,FLT3 inhibitor|JAK inhibitor
1,BRD-K12502280-001-11-4,1.000000,5.000004,0.000021,4.672645,True,True,compound_A549_long,compound,A549,long,48,TG-101348,FLT3 inhibitor|JAK inhibitor
2,BRD-K12502280-001-11-4,1.000000,5.000004,0.000030,4.527120,True,True,compound_U2OS_short,compound,U2OS,short,24,TG-101348,FLT3 inhibitor|JAK inhibitor
3,BRD-K12502280-001-11-4,1.000000,5.000004,0.000032,4.492007,True,True,compound_U2OS_long,compound,U2OS,long,48,TG-101348,FLT3 inhibitor|JAK inhibitor
4,BRD-K13094524-001-04-2,0.840278,4.094814,0.000216,3.665998,True,True,compound_A549_short,compound,A549,short,24,PFI-1,bromodomain inhibitor
5,BRD-K13094524-001-04-2,1.000000,5.000004,0.000021,4.672645,True,True,compound_A549_long,compound,A549,long,48,PFI-1,bromodomain inhibitor
6,BRD-K13094524-001-04-2,0.298359,1.287362,0.063666,1.196092,False,False,compound_U2OS_short,compound,U2OS,short,24,PFI-1,bromodomain inhibitor
7,BRD-K13094524-001-04-2,0.520889,2.389472,0.007899,2.102408,True,True,compound_U2OS_long,compound,U2OS,long,48,PFI-1,bromodomain inhibitor


In [10]:
all_brd4 = metadata.query('pert_iname=="PFI-1" or pert_iname=="TG-101348" or pert_iname=="BI-2536"')

In [11]:
replicate_retrieval_all_brd4 = pd.read_csv("output/cellprofiler_replicability_map.csv").merge(
    all_brd4, on="Metadata_broad_sample"
)
replicate_retrieval_all_brd4

,Metadata_broad_sample,mean_average_precision,nlog10pvalue,corrected_p_value,nlog10correctedpvalue,above_p_threshold,above_corrected_p_threshold,Description,Modality,Cell,time,timepoint,pert_iname,moa_list
0,BRD-K12502280-001-11-4,1.000000,5.000004,0.000033,4.482766,True,True,compound_A549_short,compound,A549,short,24,TG-101348,FLT3 inhibitor|JAK inhibitor
1,BRD-K12502280-001-11-4,1.000000,5.000004,0.000021,4.672645,True,True,compound_A549_long,compound,A549,long,48,TG-101348,FLT3 inhibitor|JAK inhibitor
2,BRD-K12502280-001-11-4,1.000000,5.000004,0.000030,4.527120,True,True,compound_U2OS_short,compound,U2OS,short,24,TG-101348,FLT3 inhibitor|JAK inhibitor
3,BRD-K12502280-001-11-4,1.000000,5.000004,0.000032,4.492007,True,True,compound_U2OS_long,compound,U2OS,long,48,TG-101348,FLT3 inhibitor|JAK inhibitor
4,BRD-K13094524-001-04-2,0.840278,4.094814,0.000216,3.665998,True,True,compound_A549_short,compound,A549,short,24,PFI-1,bromodomain inhibitor
5,BRD-K13094524-001-04-2,1.000000,5.000004,0.000021,4.672645,True,True,compound_A549_long,compound,A549,long,48,PFI-1,bromodomain inhibitor
6,BRD-K13094524-001-04-2,0.298359,1.287362,0.063666,1.196092,False,False,compound_U2OS_short,compound,U2OS,short,24,PFI-1,bromodomain inhibitor
7,BRD-K13094524-001-04-2,0.520889,2.389472,0.007899,2.102408,True,True,compound_U2OS_long,compound,U2OS,long,48,PFI-1,bromodomain inhibitor
8,BRD-K64890080-001-02-1,1.000000,5.000004,0.000033,4.482766,True,True,compound_A549_short,compound,A549,short,24,BI-2536,PLK inhibitor
9,BRD-K64890080-001-02-1,1.000000,5.000004,0.000021,4.672645,True,True,compound_A549_long,compound,A549,long,48,BI-2536,PLK inhibitor
